In [4]:
import pathlib
import shutil
import random
import os


dataset1 = pathlib.Path(
    r"C:\Users\86183\Desktop\final\uva-machine-learning-25f-projects\team-30\data\original_data\edible-and-poisonous-fungi\versions\1"
)

dataset2 = pathlib.Path(
    r"C:\Users\86183\Desktop\final\uva-machine-learning-25f-projects\team-30\data\original_data\image-mushroom-dataset\versions\1\image-mushroom-dataset"
)
target_root = pathlib.Path(
    r"C:\Users\86183\Desktop\final\uva-machine-learning-25f-projects\team-30\data\cleaned"
)

train_ratio = 0.8 
img_exts = {".jpg", ".jpeg", ".png"}
random.seed(42)



folders = [
    target_root / "train" / "edible",
    target_root / "train" / "poisonous",
    target_root / "test" / "edible",
    target_root / "test" / "poisonous"
]
for folder in folders:
    folder.mkdir(parents=True, exist_ok=True)
 


def get_class_imgs(src_folder, class_names):
   
    imgs = []
    for cls in class_names:
       
        for seg_sub in src_folder.rglob(cls):
            if seg_sub.is_dir():
                imgs.extend([f for f in seg_sub.iterdir() if f.suffix.lower() in img_exts])
    return imgs


def split_copy(imgs, target_train, target_test):
    
    random.shuffle(imgs)
    split_idx = int(len(imgs)*train_ratio)
  
    for idx, img in enumerate(imgs[:split_idx]):
        shutil.copy(img, target_train / f"{idx}_{img.name}")
  
    for idx, img in enumerate(imgs[split_idx:]):
        shutil.copy(img, target_test / f"{idx}_{img.name}")
    return len(imgs[:split_idx]), len(imgs[split_idx:])



edible_cls = ["edible mushroom sporocarp", "edible sporocarp"]
poisonous_cls = ["poisonous mushroom sporocarp", "poisonous sporocarp"]


dataset1_edible = get_class_imgs(dataset1, edible_cls)
dataset1_poison = get_class_imgs(dataset1, poisonous_cls)

dataset2_edible = get_class_imgs(dataset2, edible_cls)
dataset2_poison = get_class_imgs(dataset2, poisonous_cls)


all_edible = dataset1_edible + dataset2_edible
all_poison = dataset1_poison + dataset2_poison



edible_train_cnt, edible_test_cnt = split_copy(
    all_edible,
    target_root / "train" / "edible",
    target_root / "test" / "edible"
)


poison_train_cnt, poison_test_cnt = split_copy(
    all_poison,
    target_root / "train" / "poisonous",
    target_root / "test" / "poisonous"
)


print(f"  edible：{len(all_edible)} → train:{edible_train_cnt} / test:{edible_test_cnt}")
print(f"  poisonous：{len(all_poison)} → train:{poison_train_cnt} / test:{poison_test_cnt}")



  edible：2360 → train:1888 / test:472
  poisonous：4467 → train:3573 / test:894
